In [10]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [11]:
def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = "" + current_title
            if "section" in metadata:
                full_title += f" / {metadata['section']}"
            if "subsection" in metadata:
                full_title += f" / {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" / {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" / {metadata['subsubsubsection']}"

            content = f"[section_path]: {full_title}\n\n{doc.page_content}"
            doc = Document(page_content=content, metadata={
                **doc.metadata,
                "type": "documentation",
                "source": "dev_center_guide_allmd.md",
                "chunk_idx": chunk_idx
            })

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
# allmd 적용시 검색 품질의 급격한 저하 확(495 chunks)
str_md_file = load_markdown_file("data/dev_center_guide_allmd_touched.md") 
docs_markdown = hierarchical_markdown_split(str_md_file)

print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")

마크다운 문서 분할 완료: 495개 청크


In [21]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'PNS' in doc.page_content:
        # print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"--- doc_index: {cnt} ---")
        print(f"{doc.page_content}[EOD]")  # 처음 200자만 출력
        # print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'PNS' 문자열을 포함하는 문서 개수: {cnt}")
        
        

--- doc_index: 0 ---
[section_path]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download](v21/sample)
* [11. V21로 원스토어 인앱결제 업그레이드 하기](v21/upgrade)
* [12. Unity에서 

In [22]:
# fixed_model_name = "deepseek-coder:6.7b"
# fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"
# fixed_model_name = "llama3:8b"
fixed_model_name = "bge-m3:latest"    
# fixed_model_name = "solar:latest"

def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown #+ docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
#print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v10_1_" + fixed_model_name[:3]
os.makedirs(output_dir, exist_ok=True)
embed_and_save(total_docs, output_dir)

총 495개의 문서를 생성하였습니다.
- 마크다운 문서: 495개
✅ 임베딩 저장 완료: models/faiss_vs_rag_iap_v10_1_bge


In [26]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever

# fixed_model_name_ret = "bge-m3:latest"
# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 70, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs_markdown,
    bm25_params={"k1": 1.5, "b": 0.75}
)

bm25.k = 20

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)


# res = ensembled_retriever.invoke(
#     "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
# )

res = retriever.invoke(
    "원스토어 인앱결제의 PNS의 개념을 설명해주세요",
    # "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
)

# res = bm25.invoke(
#     # "원스토어 인앱결제의 PNS의 개념을 설명해주세요",
#     "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

검색된 문서 수: 30
--- doc_index: 0 ---
[section_path]: 07. PNS(Payment Notification Service) 이용하기 / **개요**

PNS는 Payment Notification Service의 약자입니다. PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 원스토어의 서버가 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하여 결제 트랜젝션의 상태를 손실없이 알려주기 위한 용도의 기능입니다. 즉, 개발사가 지정한 서버에서 원스토어가 정의한 규칙에 맞추어 API를 구현하면 해당 API를 원스토어의 결제 담당 서버에서 호출하는 형태입니다.  
Server to Server, 즉 서버간에 데이터를 전송한다고 할지라도 네트워크 문제로 메세지 전송 실패가 발생하기 때문에 200 OK로 응답을 인지하지 못할 경우 반복하여 메시지가 전송될 수 있습니다. 개발사의 서버는 메시지를 수신후 정의된 응답을 하여야 원스토어는 개발사 서버가 정상적으로 메시지를 전달 받았음을 인지합니다.  
결제 트렌젝션의 유형에 따라 아래와 같은 유형의 메세지가 전송됩니다.  
* 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service)&#x20;
* 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service)&#x20;  
{% hint style="warning" %}
Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로, notification 수신을 기준으로 상품(서비스)을 제공하는 것은 권장하지 않습니다.&#x20;  
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.  
원스토어는 검증 및 모니터링